In [ ]:
#Backend

In [ ]:
!pip install flask flask-cors numpy pandas librosa praat-parselmouth pydub scikit-learn joblib soundfile

In [ ]:
import os
import uuid
import pandas as pd
import numpy as np
import librosa
import parselmouth
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.utils import secure_filename
from pydub import AudioSegment
import joblib

In [ ]:

# ------------------- CONFIG -------------------
UPLOAD_DIR = os.path.join(os.getcwd(), "Uploads")
os.makedirs(UPLOAD_DIR, exist_ok=True)

ALLOWED_EXTS = {"webm", "mp4", "m4a", "mp3", "ogg", "wav"}

MODEL_PATH = "model.pkl"
bundle = joblib.load(MODEL_PATH)
pipe = bundle["pipeline"]
feature_order = bundle["feature_cols"]

In [ ]:
app = Flask(__name__)
CORS(app)

In [ ]:
# ------------------- HELPERS -------------------
def allowed_file(filename):
    return "." in filename and filename.rsplit(".", 1)[1].lower() in ALLOWED_EXTS

In [ ]:
from pydub import AudioSegment
import os
import uuid

def convert_to_wav(src_path, dst_dir):
    """
    Convert any supported audio file to 16 kHz mono PCM WAV.
    Requires ffmpeg installed and accessible in PATH.

    src_path: path to the uploaded file (any format)
    dst_dir: directory to save the converted wav file
    returns: absolute path to the converted .wav file
    """
    # generate a random name for the output
    dst_path = os.path.join(dst_dir, f"voice_{uuid.uuid4().hex}.wav")

    # load with pydub (auto-detects format using ffmpeg)
    audio = AudioSegment.from_file(src_path)

    # standardize format: mono, 16 kHz, 16-bit PCM
    audio = audio.set_channels(1).set_frame_rate(16000).set_sample_width(2)

    # export to WAV
    audio.export(dst_path, format="wav")

    return dst_path


In [ ]:
def _safe(arr, fn, default=0.0):
    arr = np.asarray(arr, dtype=float)
    arr = arr[~np.isnan(arr)]
    return float(fn(arr)) if arr.size else float(default)

def extract_pd_base_features(audio_path, sr_target=16000):
    y, sr = librosa.load(audio_path, sr=sr_target, mono=True)
    y, _ = librosa.effects.trim(y)
    if len(y) < 0.3 * sr:
        raise ValueError("Audio too short. Record at least 3 seconds.")

    f0, _, _ = librosa.pyin(y, fmin=50, fmax=400, sr=sr)
    f0v = f0[~np.isnan(f0)] if f0 is not None else np.array([])

    snd = parselmouth.Sound(y, sr)
    pp  = parselmouth.praat.call(snd, "To PointProcess (periodic, cc)", 75, 500)

    jitt_pct = parselmouth.praat.call(pp, "Get jitter (local)", 0, 0, 75, 500, 1.3)
    jitt_abs = parselmouth.praat.call(pp, "Get jitter (local, absolute)", 0, 0, 75, 500, 1.3)
    jitt_rap = parselmouth.praat.call(pp, "Get jitter (rap)", 0, 0, 75, 500, 1.3)
    jitt_ppq = parselmouth.praat.call(pp, "Get jitter (ppq5)", 0, 0, 75, 500, 1.3)
    jitt_ddp = 3.0 * jitt_rap

    shim_loc    = parselmouth.praat.call([snd, pp], "Get shimmer (local)", 0, 0, 75, 500, 1.3, 1.6)
    shim_loc_db = parselmouth.praat.call([snd, pp], "Get shimmer (local_dB)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq3   = parselmouth.praat.call([snd, pp], "Get shimmer (apq3)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq5   = parselmouth.praat.call([snd, pp], "Get shimmer (apq5)", 0, 0, 75, 500, 1.3, 1.6)
    shim_apq11  = parselmouth.praat.call([snd, pp], "Get shimmer (apq11)", 0, 0, 75, 500, 1.3, 1.6)
    shim_dda    = 3.0 * shim_apq3

    harm = parselmouth.praat.call(snd, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
    hnr = parselmouth.praat.call(harm, "Get mean", 0, 0)
    nhr = float(10 ** (-hnr/10.0)) if np.isfinite(hnr) else 0.0

    return {
        "MDVP:Fo(Hz)": _safe(f0v, np.mean),
        "MDVP:Fhi(Hz)": _safe(f0v, np.max),
        "MDVP:Flo(Hz)": _safe(f0v, np.min),
        "MDVP:Jitter(%)": jitt_pct,
        "MDVP:Jitter(Abs)": jitt_abs,
        "MDVP:RAP": jitt_rap,
        "MDVP:PPQ": jitt_ppq,
        "Jitter:DDP": jitt_ddp,
        "MDVP:Shimmer": shim_loc,
        "MDVP:Shimmer(dB)": shim_loc_db,
        "Shimmer:APQ3": shim_apq3,
        "Shimmer:APQ5": shim_apq5,
        "MDVP:APQ": shim_apq11,
        "Shimmer:DDA": shim_dda,
        "NHR": nhr,
        "HNR": hnr
    }

In [ ]:
from werkzeug.utils import secure_filename
import uuid

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    uploaded = request.files["file"]
    if uploaded.filename == "":
        return jsonify({"error": "Empty filename"}), 400

    if not allowed_file(uploaded.filename):
        return jsonify({"error": "Unsupported file format"}), 400

    # Use a safe, unique temporary filename to avoid collisions
    ext = uploaded.filename.rsplit(".", 1)[1].lower()
    temp_name = f"upload_{uuid.uuid4().hex}.{ext}"
    temp_path = os.path.join(UPLOAD_DIR, secure_filename(temp_name))
    uploaded.save(temp_path)

    wav_path = None
    try:
        # Convert to wav (returns path)
        wav_path = convert_to_wav(temp_path, UPLOAD_DIR)

        # Extract features and predict
        features = extract_pd_base_features(wav_path)
        X = pd.DataFrame([[features.get(f, 0.0) for f in feature_order]], columns=feature_order)
        result = pipe.predict(X)[0]
        label = "Parkinson's Detected" if int(result) == 1 else "Healthy"

        return jsonify({"result": label}), 200

    except Exception as e:
        # Return helpful error for debugging (but avoid leaking secrets in prod)
        return jsonify({"error": str(e)}), 500

    finally:
        # CLEANUP: always try to remove both files if they exist
        for p in (temp_path, wav_path):
            try:
                if p and os.path.exists(p):
                    os.remove(p)
            except Exception:
                # never raise from cleanup; just swallow it
                pass


In [ ]:
!pip install pyngrok

In [ ]:
from pyngrok import ngrok
import os
from flask import Flask, jsonify, request
from flask_cors import CORS

# set your ngrok token (replace YOUR_TOKEN_HERE with yours)
os.environ["NGROK_AUTH_TOKEN"] = "2p1xj3zKO5AoIax14Ma0rMzNhaI_6DEfxNQfcejD9PfdBmSQU"
ngrok.set_auth_token(os.environ["NGROK_AUTH_TOKEN"])

# open tunnel
public_url = ngrok.connect(5000)
print("Public URL:", public_url.public_url)

@app.route("/")
def home():
    return jsonify({"message": "Backend running!"})

# run Flask app inside Colab
app.run(host="0.0.0.0", port=5000)
